In [3]:
import scipy.io as sio # python 讀取 .mat file
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
from keras.utils import np_utils
from keras import backend as keras_backend
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices()) #check available device

Using TensorFlow backend.


In [18]:
matData = sio.loadmat("fisheriris.mat") # 鳶尾花

#print(matData.items()) # 得知有哪些 items 

# iris(species, meas)
irisSpecies = np.array(matData["species"]) # type(setosa, versicolor, virginica)
irisMeas = np.array(matData["meas"]) # sepal(花萼)長、寬, petal(花瓣)長、寬 cm
print(irisSpecies[0], irisMeas[0])

[array(['setosa'], dtype='<U6')] [5.1 3.5 1.4 0.2]


In [19]:
matData = sio.loadmat("ionosphere.mat") # 電離層
#print(matData.items())

# X, Y
ionosphereX = np.array(matData["X"])
ionosphereY = np.array(matData["Y"])
print(ionosphereX[0], ionosphereY[0])

[ 1.       0.       0.99539 -0.05889  0.85243  0.02306  0.83398 -0.37708
  1.       0.0376   0.85243 -0.17755  0.59755 -0.44945  0.60536 -0.38223
  0.84356 -0.38542  0.58212 -0.32192  0.56971 -0.29674  0.36946 -0.47357
  0.56811 -0.51171  0.41078 -0.46168  0.21266 -0.3409   0.42267 -0.54487
  0.18641 -0.453  ] [array(['g'], dtype='<U1')]
